<a href="https://colab.research.google.com/github/dietmarja/LLM-Elements/blob/main/QLoRA_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QLoRA (Quantized Low-Rank Adaptation).
## QLoRA typically involves quantizing the weights of the model in addition toapplying a low-rank adaptation. For simplicity, we'll use a basic quantization approach where we scale the weights to integers and then scale them back during computation.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [3]:
# QLoRA Layer Definition
class QLoRALayer(nn.Module):
    def __init__(self, in_features, out_features, rank, quant_bits=8):
        super(QLoRALayer, self).__init__()
        self.rank = rank
        self.quant_bits = quant_bits
        self.W = nn.Linear(in_features, out_features, bias=False)
        self.A = nn.Linear(in_features, rank, bias=False)
        self.B = nn.Linear(rank, out_features, bias=False)

        # Initialize A and B with small values
        nn.init.normal_(self.A.weight, std=0.01)
        nn.init.normal_(self.B.weight, std=0.01)

    def forward(self, x):
        return self.quantize(self.W(x) + self.B(self.A(x)))

    def quantize(self, x):
        scale = (2 ** self.quant_bits - 1) / x.max()
        return torch.round(x * scale) / scale

    def print_weights(self):
        print(f"Full-rank weight matrix (W): \n{self.W.weight.data}")
        print(f"Low-rank weight matrix A: \n{self.A.weight.data}")
        print(f"Low-rank weight matrix B: \n{self.B.weight.data}")


In [4]:
# Simple Model Definition
class SimpleModel(nn.Module):
    def __init__(self, input_dim, output_dim, qlora_rank=None):
        super(SimpleModel, self).__init__()
        if qlora_rank:
            self.layer = QLoRALayer(input_dim, output_dim, qlora_rank)
        else:
            self.layer = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.layer(x)

    def print_weights(self):
        if isinstance(self.layer, QLoRALayer):
            self.layer.print_weights()
        else:
            print(f"Full-rank weight matrix (W): \n{self.layer.weight.data}")

In [5]:
# Training Function
def train_model(model, inputs, targets, epochs=100):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    losses = []
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

    return losses

In [6]:
# Evaluation Function
def evaluate_model(model, test_inputs):
    model.eval()
    with torch.no_grad():
        predictions = model(test_inputs)
        print(f'Predictions: {predictions}')

In [7]:
# Define model parameters
input_dim = 10
output_dim = 5

In [8]:
# Dummy data for demonstration
inputs = torch.randn(8, input_dim)
targets = torch.randn(8, output_dim)

In [9]:
# QLoRA ranks to evaluate
qlora_ranks = [1, 2, 3, 4]

In [10]:
# Dictionary to store all losses
all_losses = {"No QLoRA": train_model(SimpleModel(input_dim, output_dim), inputs, targets)}

Epoch [10/100], Loss: 0.7408
Epoch [20/100], Loss: 0.4374
Epoch [30/100], Loss: 0.3043
Epoch [40/100], Loss: 0.2410
Epoch [50/100], Loss: 0.2009
Epoch [60/100], Loss: 0.1714
Epoch [70/100], Loss: 0.1487
Epoch [80/100], Loss: 0.1308
Epoch [90/100], Loss: 0.1156
Epoch [100/100], Loss: 0.1023


In [11]:
# Train models with different QLoRA ranks
for rank in qlora_ranks:
    print(f"Training with QLoRA rank {rank}...")
    model_qlora = SimpleModel(input_dim, output_dim, rank)
    losses_qlora = train_model(model_qlora, inputs, targets)
    all_losses[f"QLoRA rank {rank}"] = losses_qlora
    model_qlora.print_weights()

Training with QLoRA rank 1...
Epoch [10/100], Loss: 1.4289
Epoch [20/100], Loss: 1.3887
Epoch [30/100], Loss: 1.4202
Epoch [40/100], Loss: 1.4721
Epoch [50/100], Loss: 1.5063
Epoch [60/100], Loss: 1.5068
Epoch [70/100], Loss: 1.4924
Epoch [80/100], Loss: 1.5402
Epoch [90/100], Loss: 1.5960
Epoch [100/100], Loss: 1.6197
Full-rank weight matrix (W): 
tensor([[-0.0087, -0.1495, -0.1074, -0.0533,  0.0293,  0.0983,  0.2615,  0.0471,
         -0.2149,  0.0226],
        [-0.1343, -0.4277, -0.2102, -0.0246,  0.2071,  0.3349, -0.0436,  0.0741,
          0.1004,  0.0463],
        [-0.0469,  0.1522, -0.0928,  0.2001, -0.1729,  0.1245,  0.0027,  0.0351,
         -0.1612,  0.1501],
        [-0.2156,  0.0906, -0.1339,  0.0184,  0.0485,  0.2646,  0.1240, -0.1214,
          0.0041, -0.0635],
        [-0.0921, -0.2336,  0.0981,  0.2251,  0.1633,  0.2597,  0.4212,  0.0176,
         -0.2575, -0.0151]])
Low-rank weight matrix A: 
tensor([[-0.1087, -0.1654, -0.1439,  0.3111,  0.0326,  0.2459,  0.2305,  0.0

In [ ]:
# Plot the losses for all models
plt.figure(figsize=(12, 8))
for label, losses in all_losses.items():
    plt.plot(losses, label=label)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss with and without QLoRA')
plt.legend()
plt.show()

In [13]:
# Example prediction
test_inputs = torch.randn(2, input_dim)
for rank in ["No QLoRA"] + [f"QLoRA rank {r}" for r in qlora_ranks]:
    print(f"Evaluating model with {rank}...")
    model = SimpleModel(input_dim, output_dim) if rank == "No QLoRA" else SimpleModel(input_dim, output_dim, int(rank.split()[-1]))
    evaluate_model(model, test_inputs)

Evaluating model with No QLoRA...
Predictions: tensor([[ 0.2917,  0.1646, -0.2920, -1.1431, -0.6351],
        [-0.6976,  0.0238, -0.4948, -0.2290, -0.8304]])
Evaluating model with QLoRA rank 1...
Predictions: tensor([[-0.3238,  1.1967,  0.1502,  0.2112,  0.2957],
        [ 0.3848,  0.6570, -0.7321,  0.5209, -0.7650]])
Evaluating model with QLoRA rank 2...
Predictions: tensor([[ 0.0568,  0.4424,  0.0090, -0.1285,  0.7623],
        [-0.5740, -0.2511,  0.7414,  0.5560,  0.1196]])
Evaluating model with QLoRA rank 3...
Predictions: tensor([[ 0.3698,  0.6944, -0.1524,  0.1299,  0.3783],
        [-1.0727,  0.7198, -0.3839,  0.3500,  0.2371]])
Evaluating model with QLoRA rank 4...
Predictions: tensor([[-0.5002,  0.6161,  0.0497,  0.0795, -0.0795],
        [ 0.6459,  0.8447,  0.2981, -0.5830,  0.2683]])
